In [1]:
import sys
import os
import re
import pandas as pd
import time
import psycopg2
import requests
from bs4 import BeautifulSoup

from convertPDF.webScrape.pdf2xml import *
from convertPDF.webScrape.xml2txt import *

from convertPDF.driver import parseFullDay, bulkParse

from convertPDF.infoFns.genInfoFns import parseGenInfo
from convertPDF.infoFns.horseInfoFns import parseHorseInfo
from convertPDF.infoFns.timesInfoFns import parseTimeInfo
from convertPDF.infoFns.betInfoFns import parseBetInfo
from convertPDF.infoFns.runlineInfoFns import parseRunlineInfo
from convertPDF.infoFns.endInfoFns import parseEndInfo

from populateDB import generateEntries, populateDB

from convertPDF.infoFns.regexPatterns import *

In [28]:
def renameHtml(folderAddress):
    fileNames = os.listdir(folderAddress)
    for fileName in fileNames:
        with open(folderAddress + '/' + fileName) as file:
            raw = file.read()

            soup = BeautifulSoup(raw)

            date = re.search(r', (.*)', soup.center.text.strip()).group(1)

            os.rename(folderAddress + '/' + fileName, folderAddress + '/' + date + '.html')

In [29]:
renameHtml('./../html')

In [8]:
from bs4 import BeautifulSoup

fileNames = os.listdir('./../html/')
for fileName in fileNames:
    with open('./../html//'  +fileName) as file:
        raw = file.read()

        soup = BeautifulSoup(raw)

            

In [30]:
os.rename('./../html//' + fileName, './../html/' + soup.center.text.strip() + '.html')

In [19]:
soup.center.text.strip()

'Tuesday, January 7 2020'

In [27]:
date = re.search(, soup.center.text.strip())
date

<re.Match object; span=(0, 1), match='T'>

In [11]:
jack = requests.get('https://www.equibase.com/premium/eqpVchartBuy.cfm?mo=1&da=1&yr=2019&trackco=ALL;ALL&cl=Y').text

In [12]:
jack

'<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript" src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3"></script><script src="/inan-giues-thing-starra-your-Thange-Macb-This-br" async></script></head><body style="margin:0px;height:100%"><iframe id="main-iframe" src="/_Incapsula_Resource?SWUDNSAI=30&xinfo=12-167087838-0%20NNNY%20RT%281624594933514%20266%29%20q%280%20-1%20-1%200%29%20r%281%20-1%29%20B12%2814%2c0%2c0%29%20U18&incident_id=1363000130394797261-883085399731405644&edet=12&cinfo=0e000000ce42&rpinfo=0&cts=GpY%2bkoLcL6zX0Y%2b0YHO0VXwOs%2buIcO4XCbQJOH7p8gql58FpvF9jL5BEBqJfhEgS&mth=GET" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 1363000130394797261-883085399731405644

In [2]:
start = time.time()
entries = generateEntries('./../charts/txts')
end = time.time()

end - start

No betting detected for race.
No betting detected for race.
No betting detected for race.
No betting detected for race.
No betting detected for race.
No betting detected for race.
No betting detected for race.
No betting detected for race.
Error in parseBetLine on line:  $2.00 Daily Double 2-9 17.00 3,468

No betting detected for race.
No betting detected for race.
No betting detected for race.
Match error in parseWeatherConditions on line:  Weather: Cloudy Track:

Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

No betting detected for race.
Match error in parseWeatherConditions on line:  Weather: Cloudy Track:

Match error in parseStart on line:  Off at: : Start: Good for all except

Match error in parseFractionalTimes on line:  Final Time:

No betting detected for race.
Match error in parseWeatherConditions on line:  Weather: Cloudy Track:

Match error in parseStart on line:  Off at: : Start: Good fo

8.91314172744751

In [3]:
conn = psycopg2.connect(
    host = "localhost",
    database = "horses",
    user = "karisch",
    password = "cocacola",
    port = 5432
)

In [4]:
populateDB(conn, entries)

In [5]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM main.races;")
    races = cur.fetchall()
    cur.execute("SELECT * FROM main.horses;")
    horses = cur.fetchall()

In [6]:
raceCols = (
            "track", 
            "date",
            "race",
            "distance",
            "surface",
            "weather",
            "conditions",
            "startTime",
            "startNote",
            "segment1",
            "segment2",
            "segment3",
            "segment4",
            "segment5",
            "segments",
            "fracTime1",
            "fracTime2",
            "fracTime3",
            "fracTime4",
            "fracTime5",
            "finalTime",
            "runup",
            "wpsPool",
            "firstPlaceWin",
            "firstPlacePlace",
            "firstPlaceShow",
            "secondPlacePlace",
            "secondPlaceShow",
            "thirdPlaceShow",
            "exactaBuyin",
            "exactaFinish",
            "exactaPayout",
            "exactaPool",
            "trifectaBuyin",
            "trifectaFinish",
            "trifectaPayout",
            "trifectaPool"
        )

In [7]:
horseCols = (
                "track",
                "date",
                "race",
                "horseProgram",
                "horseName",
                "lastRaceDay",
                "lastRaceMonth",
                "lastRaceYear",
                "lastRaceTrack",
                "jockey",
                "weight",
                "ME",
                "placePP",
                "placeSeg1",
                "placeSeg2",
                "placeSeg3",
                "placeSeg4",
                "placeSeg5",
                "placeSeg6",
                "odds",
                "comments",
                "lastRaceNum",
                "lastRacePlace",
                "lengthsSeg1",
                "lengthsSeg2",
                "lengthsSeg3",
                "lengthsSeg4",
                "lengthsSeg5",
                "lengthsSeg6",
                "rlLengthsSeg1",
                "rlLengthsSeg2",
                "rlLengthsSeg3",
                "rlLengthsSeg4",
                "rlLengthsSeg5",
                "rlLengthsSeg6",
                "rlPlaceSeg1",
                "rlPlaceSeg2",
                "rlPlaceSeg3",
                "rlPlaceSeg4",
                "rlPlaceSeg5",
                "rlPlaceSeg6",
                "trainer",
                "owner"
                )

In [8]:
racesdf = pd.DataFrame(races, columns=raceCols)
horsesdf = pd.DataFrame(horses, columns=horseCols)

In [9]:
racesdf.to_csv('./../outputs/races.csv')
horsesdf.to_csv('./../outputs/horses.csv')

In [16]:
conn.close()

In [15]:
conn.commit()

In [10]:
with open('./../charts/txts/eqbPDFChartPlus - 2021-06-10T234034.773.txt') as file:
    full = file.readlines()
    jack = parseFullDay(full)

No betting detected for race.


In [12]:
for line in full:
    if re.search('290.70', line) is not None:
        print(line)

 2 Sunny Puzzle $2.00 Pick 5 6-1-6-10-6 (5 correct) 290.70 20,221



In [17]:
pattern = r'^ \d?\d[ABCX]? [^0-9]+ (\d?\d?\d\.\d\d)()()(.*)$'
re.search(pattern, line)

In [2]:
line = ' 2 Sunny Puzzle $2.00 Pick 5 6-1-6-10-6 (5 correct) 290.70 20,221'
from convertPDF.betInfoFns import parseBetLine

parseBetLine(line)

['', '', '', '$2.00 Pick 5 6-1-6-10-6 (5 correct) 290.70 20,221']